## ComfyUI

### Install

In [ ]:
#@title Environment Setup

OPTIONS = {}

UPDATE_COMFY_UI = True  #@param {type:"boolean"}
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

from google.colab import drive
drive.mount('/content/drive')
install_root = "/content/drive/MyDrive/ai"
%mkdir -p $install_root
%cd $install_root

import os
if not os.path.exists(f"{install_root}/comfyui"):
    !git clone https://github.com/comfyanonymous/ComfyUI comfyui
    !wget -q https://github.com/brenton-thomas/ComfyUI-Colab/raw/main/extra_model_paths.yaml -O extra_model_paths.yaml

%cd "$install_root/comfyui"

if OPTIONS['UPDATE_COMFY_UI']:
  !git reset --hard
  !git pull origin master
  !rm -f extra_model_paths.yaml
  !wget -q https://github.com/brenton-thomas/ComfyUI-Colab/raw/main/extra_model_paths.yaml -O extra_model_paths.yaml



import ipywidgets as widgets
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
inf('\u2714 Done','success', '50px')

### Run with Cloudflared

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request


from google.colab import drive
drive.mount('/content/drive')
install_root = "/content/drive/MyDrive/ai"
%mkdir -p $install_root

#todo: need a more graceful way to exit at some point
if not os.path.exists(f"{install_root}/comfyui"):
    raise KeyboardInterrupt

%cd "$install_root/comfyui"

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run with Local Tunnel




In [ ]:
### workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

import subprocess
import threading
import time
import socket
import urllib.request

!npm install -g localtunnel

from google.colab import drive
drive.mount('/content/drive')
install_root = "/content/drive/MyDrive/ai"
%mkdir -p $install_root

#todo: need a more graceful way to exit at some point
if not os.path.exists(f"{install_root}/comfyui"):
    raise KeyboardInterrupt

%cd "$install_root/comfyui"

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117


def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server